In [1]:
# 
import numpy as np
import cv2 # opencv-python which is an Open Source Computer Vision Library
import sys
import os
import matplotlib.pyplot as plt # matplotlib.pyplot which provides a MATLAB-like plotting framework

def close_window():
    cv2.destroyAllWindows()  # Close the window
    cv2.waitKey(1)
    cv2.waitKey(1)
    
def ifNotExistExit(filename):
    if not filename:
        print('filename is None')
        sys.exit()

In [3]:
## Hough line detection
src = cv2.imread('./fig/checkerboard.png', cv2.IMREAD_GRAYSCALE)
if src is None:
    print('Image load failed!')
    sys.exit()

edges = cv2.Canny(src, 50, 150)
lines = cv2.HoughLinesP(edges, 1, np.pi / 360, 60, minLineLength=10, maxLineGap=20) # cv2.CV_32F
print('lines.shape:', lines.shape) # (n, 1, 2)

dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

for i in range(lines.shape[0]):
    pt1 = (lines[i][0][0], lines[i][0][1])
    pt2 = (lines[i][0][2], lines[i][0][3])
    cv2.line(dst, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(dst, str(i), pt1, cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('edges', edges)
cv2.imshow("dst", dst)

cv2.waitKey()
close_window()

lines.shape: (109, 1, 4)


In [2]:
#
## 연결 객체 검출: 레이블링
src = cv2.imread('./fig/keyboard.jpg')
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# mask 하는 이유는 흰색을 255(컴퓨터는 255를 물채로 인식)로 하고 
# 검은색을 0으로 하기 위해서임
_, mask = cv2.threshold(src_gray, 120, 255, cv2.THRESH_BINARY_INV) 
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

# cnt
print('cnt:', cnt)

for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]
    if area <= 200:
        continue
    cv2.rectangle(src, (x, y, w, h), (0, 0, 255), 2)
    cv2.putText(src, str(i), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

cv2.imshow('src', src)
cv2.imshow('mask', mask)
cv2.imshow('src_gray', src_gray)

cv2.waitKey()
close_window()


cnt: 124


2025-03-25 19:15:48.351 python[15042:629397] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-25 19:15:48.351 python[15042:629397] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## 외곽선 검출

In [ ]:
## 외곽선 검출 from ./fig/shapes.png
src = cv2.imread('./fig/shape.png')

if src is None:
    print('Image load failed!')
    sys.exit()


# computer regcognize white color as 255 and black color as 0
# so, object should be white and background should be black
# (This is opposite to human's eye)
# So, we need to convert the image to binary image
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
_, src_bin = cv2.threshold(src_gray, 150, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)


# cv2.RETR_EXTERNAL: 외곽선 중 가장 바깥쪽의 외곽선만 찾음
# cv2.CHAIN_APPROX_NONE: 외곽선을 구성하는 모든 점을 반환
# contours: 외곽선 좌표
# _ : 계층 정보
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
dst = cv2.cvtColor(src_bin, cv2.COLOR_GRAY2BGR)

for i in range(len(contours)):    
    cv2.drawContours(dst, contours, i, (255, 0, 0), 2) # blue
    cv2.putText(dst, str(i), tuple(contours[i][0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 1, cv2.LINE_AA) # red
    
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)

cv2.waitKey()
close_window()

## 동영상 처리

In [3]:
#
cap = cv2.VideoCapture('./fig/video/PETS2000.avi')
if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

# get the first frame as the background
ret, back = cap.read()
if not ret:
    print('Background image registration failed!')
    sys.exit()

back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY) # convert to gray scale
    
cap.isOpened()

while True:
    ret, frame = cap.read() # ret: True or False, frame: image data (from the first frame to the last frame)
    
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, back) # difference between the current frame and the background
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY) # thresholding
    diff_red = cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR) # convert to color image
    diff_red[:,:,2] = 0
    
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(diff)
    for i in range(1, cnts):
        (x, y, w, h, area) = stats[i]
        if area < 100:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
        cv2.putText(frame, str(i), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1)
    
    cv2.imshow('frame', frame)
    cv2.imshow('diff', diff_red) 
    
    if cv2.waitKey(10) == 27: # ESC key
        break

cap.release()
cv2.waitKey() # wait until any key,'q' is pressed
close_window()


2025-03-25 21:31:02.888 python[20754:818538] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-25 21:31:02.888 python[20754:818538] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
# Webcam (macbook pro Intel cpu)
# WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Camera open failed!')
    sys.exit()

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    edge = cv2.Canny(frame, 50, 150)
    frame_gaussian = cv2.GaussianBlur(frame, (0, 0), 2)
    frame_gray = cv2.cvtColor(frame_gaussian, cv2.COLOR_BGR2GRAY)
    frame_gray_inversed = cv2.bitwise_not(frame_gray)
    edge_gaussian = cv2.Canny(frame_gaussian, 50, 150)
        
    cv2.imshow('frame', frame)
    cv2.imshow('edge', edge)
    cv2.imshow('frame_gaussian', frame_gaussian)
    cv2.imshow('frame_gray', frame_gray)
    cv2.imshow('frame_gray_inversed', frame_gray_inversed)
    cv2.imshow('edge_gaussian', edge_gaussian)
    
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.waitKey()
close_window()



## 기하학적 모멘트

In [ ]:
## 기하학적 모멘트 (Hu moment)
obj = cv2.imread('./fig/spades.png', cv2.IMREAD_GRAYSCALE)
src = cv2.imread('./fig/symbols.png', cv2.IMREAD_GRAYSCALE)

_, obj_bin = cv2.threshold(obj, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

obj_pts = obj_contours[0]

_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
src_contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for pts in src_contours:
    if cv2.contourArea(pts) < 1000:
        continue
    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255, 0, 0), 1)
    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)
    cv2.putText(dst, str(round(dist, 3)), (rc[0], rc[1] - 3), cv2.FONT_HERSHEY_COMPLEX, 0.8,
                (255, 0, 0), 1, cv2.LINE_AA)
    
    if dist < 0.1:
        cv2.rectangle(dst, rc, (0, 0, 255), 2)
        cv2.putText(dst, str(round(dist, 3)), (rc[0], rc[1] - 3), cv2.FONT_HERSHEY_COMPLEX, 0.8,
                (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('dst', dst)
cv2.imshow('obj_bin', obj_bin)
cv2.imshow('src_bin', src_bin)

cv2.waitKey()
close_window()